In [1]:
import pandas as pd
from transformers import pipeline

# Mapea los valores de texto a valores numéricos
mapeo_valores = {'1 star': 1, '2 stars': 2, '3 stars': 3, '4 stars': 4, '5 stars': 5}


csv_path = 'comments/tutivargasm_comments.csv'

perfil = 'belenegri'
data = pd.read_csv(csv_path)

data.head()

C:\Users\samue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,post_id,comment_id,comment_user,comment_text
0,3214913405461494889,17967978077628137,pameu3,💙🩵🙌🏼
1,3214913405461494889,18043177354530636,linammina,Wowwww!!! Q lugar más precioso!
2,3214913405461494889,17991100607148280,lamalamexicana,"Wow, están preciosas tus fotos ❤️❤️❤️ Hermoso ..."
3,3214913405461494889,18041465953542740,diego.alexande,Hermoso paisaje
4,3214913405461494889,17977960481339702,valentinapastrana,De donde es tu overall? 😻😻


In [2]:
# Crear un pipeline de análisis de sentimiento
pipe = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

# Obtener la lista de comentarios y post_ids
df_list = data['comment_text'].tolist()
post_ids = data['post_id'].tolist()

In [3]:
max_comment_length = 512
for post_id, comment in zip(post_ids, df_list):
        if isinstance(comment, str) and comment.strip():  # Verificar si el comentario no está vacío
            comment = comment[:max_comment_length] #truncar el comentario si es mas largo que la longitud maxima
            result = pipe(comment)

            print(f"Post ID: {post_id}")
            print(f"Comentario: {comment}")
            print(f"Polaridad: {result[0]['label']}, Score: {round(result[0]['score'], 4)}")
            print()

            # Agregar los resultados al DataFrame
            data.loc[data['post_id'] == post_id, 'sentiment_label'] = mapeo_valores[result[0]['label']]

        elif isinstance(comment, str) and not comment.strip():
            print(f"Post ID: {post_id}")
            print("Comentario vacío. No se realiza el análisis de sentimiento.")
            print()
        else:
            print(f"Post ID: {post_id}")
            print("Comentario no es una cadena de texto. No se realiza el análisis de sentimiento.")
            print()


Post ID: 3214913405461494889
Comentario: 💙🩵🙌🏼
Polaridad: 5 stars, Score: 0.3086

Post ID: 3214913405461494889
Comentario: Wowwww!!! Q lugar más precioso!
Polaridad: 5 stars, Score: 0.9051

Post ID: 3214913405461494889
Comentario: Wow, están preciosas tus fotos ❤️❤️❤️ Hermoso lugar 🧗‍♀️
Polaridad: 5 stars, Score: 0.9268

Post ID: 3214913405461494889
Comentario: Hermoso paisaje
Polaridad: 5 stars, Score: 0.7544

Post ID: 3214913405461494889
Comentario: De donde es tu overall? 😻😻
Polaridad: 1 star, Score: 0.3287

Post ID: 3214913405461494889
Comentario: Que belleza mi amor ❤️❤️
Polaridad: 5 stars, Score: 0.6832

Post ID: 3214913405461494889
Comentario: Me encanta pero me da como miedo viajar allá con tanta mal noticia de México 😢
Polaridad: 4 stars, Score: 0.3659

Post ID: 3214913405461494889
Comentario: Demasiadoooo😍😍😍
Polaridad: 5 stars, Score: 0.3086

Post ID: 3214913405461494889
Comentario: Qué maravilla🫀
Polaridad: 5 stars, Score: 0.2433

Post ID: 3214913405461494889
Comentario: Qué 

In [ ]:
#  Calcular la media de los resultados por post_id
csv_sentiments = data.groupby('post_id')['sentiment_label'].mean().round().reset_index()

# Exportar la media de los resultados a un archivo Excel
csv_sentiments.to_csv(f'sentiments/{perfil}_sentiments.csv', index=False)

csv_sentiments

,post_id,sentiment_label
0,2062791437233396551,5.0
1,2097568187229975930,1.0
2,2120739426077499911,5.0
3,2135314990125799242,5.0
4,2183846987579986109,5.0
...,...,...
89,3099963399278197762,5.0
90,3108644839294741594,5.0
91,3115165281417420564,5.0
92,3207294882905542946,5.0
